In [1]:
# Import the required library
import requests
from ipywidgets import widgets, VBox, HBox, Output
from IPython.display import display

# Function to trace URL redirects and display detailed HTTP info
def trace_url_jumps_with_details(initial_url, proxy=None):
    try:
        # Configure proxy if provided
        proxies = {"http": proxy, "https": proxy} if proxy else None

        # Make a GET request with proxy and enable redirect tracking
        response = requests.get(initial_url, allow_redirects=True, proxies=proxies)

        if response.history:
            print(f"\n🔍 Tracing URL redirects for: {initial_url}")
            print("\n--------------------------------------------------\n")
            for i, resp in enumerate(response.history, start=1):
                print(f"➡️  **Hop #{i}**")
                print(f"🌐 Request URL: {resp.url}")
                print(f"📄 Status Code: {resp.status_code} {'✅ Success' if resp.status_code == 200 else '🔄 Redirect'}")
                print("📬 Headers:")
                for key, value in resp.headers.items():
                    print(f"  📎 {key}: {value}")
                print("📝 Response body (limit 200 chars):")
                print(resp.text[:200] if resp.text else "⚠️ (No body)")
                print("\n--------------------------------------------------\n")

        # Final response (landing page)
        print("🎯 **Final Destination:**")
        print(f"🌐 URL: {response.url}")
        print(f"📄 Status Code: {response.status_code} {'✅ Success' if response.status_code == 200 else '⚠️ Warning'}")
        print("📬 Headers:")
        for key, value in response.headers.items():
            print(f"  📎 {key}: {value}")
        print("📝 Response body (limit 200 chars):")
        print(response.text[:200] if response.text else "⚠️ (No body)")
        print("🎉 Inspection Complete!")

        return response.url  # Return the final destination URL as the result

    except requests.exceptions.RequestException as e:
        # Handle potential request errors (e.g., invalid links, connectivity, or proxy issues)
        print(f"❌ An error occurred: {e}")
        return None

###################################
# CLICK EVENT HANDLERS FOR BUTTONS #
###################################
def on_click_trace(b):
    with output:  # Redirect all output to the Output widget
        output.clear_output()  # Clear previous output in the widget
        # Getting user input from the text box
        url_to_trace = user_url.value.strip()

        # Optional proxy input from user
        proxy = None  # For simplicity, proxy input is skipped in this GUI version

        # Validate input
        if not url_to_trace.startswith(("http://", "https://")):
            print("⚠️ Error: Please ensure the URL starts with `http://` or `https://`. Example: https://example.com")
        else:
            # Call the trace function and display the results
            final_url = trace_url_jumps_with_details(url_to_trace, proxy=proxy)
            if final_url:
                print(f"\n✅ The final resolved URL is: {final_url}")

def on_click_clear(b):
    with output:
        output.clear_output()  # Clear the Output widget's content

######################
# GUI IMPLEMENTATION #
######################
# Widgets
user_url = widgets.Text(
    placeholder='Enter the URL to trace',
    description='URL:',
    layout=widgets.Layout(width='600px')
)

output = Output()

# Buttons for services
run_button = widgets.Button(description='Run', button_style='success')
clear_button = widgets.Button(description='Clear Output', button_style='danger')

# Button event bindings
run_button.on_click(on_click_trace)
clear_button.on_click(on_click_clear)

##############
# APP LAYOUT #
##############
app_layout = VBox([
    widgets.HTML("<h2>URL Hops Inspector 🚀🔍</h2>"),
    user_url,
    HBox([run_button, clear_button]),
    output
])

# Display the App
display(app_layout)